In [1]:
import pymysql
import pandas as pd

In [6]:
# Create a function for easily querying data.
def qry(sql):

    # Open database connection
    connection = pymysql.connect("localhost","root","rootroot","sakila_snowflake" ) # Notice sakila is the chosen schema
    df = pd.read_sql(sql, connection)

    # disconnect from server
    connection.close()
    
    # return data.
    return df

In [7]:
#Q2 a)
sql = "show tables;"
qry(sql)

,Tables_in_sakila_snowflake
0,dim_actor
1,dim_customer
2,dim_date
3,dim_film
4,dim_film_actor_bridge
5,dim_location_address
6,dim_location_city
7,dim_location_country
8,dim_staff
9,dim_store


In [ ]:
# Q2 b) DDL
db = pymysql.connect("localhost","root","rootroot","sakila" )
# prepare a cursor object using cursor() method
cursor = db.cursor()
# execute SQL query using execute() method.
sql ='''CREATE TABLE IF NOT EXISTS  `sakila_snowflake`.`fact_rental`(
  `rental_id` INT(10) NOT NULL,
  `rental_last_update` TIMESTAMP NOT NULL,
  `customer_key` INT(8) NOT NULL,
  `staff_key` INT(8) NOT NULL,
  `film_key` INT(8) NOT NULL,
  `store_key` INT(8) NOT NULL,
  `rental_date_key` BIGINT(20) NOT NULL,
  `return_date_key` BIGINT(20) NULL,
  `count_returns` INT(10) NOT NULL,
  `count_rentals` INT(10) NOT NULL,
  `rental_duration` INT(10) NULL,
  `dollar_amount` FLOAT NULL,
PRIMARY KEY (`rental_id`),
  INDEX `dim_store_fact_rental_fk` (`store_key` ASC),
  INDEX `dim_staff_fact_rental_fk` (`staff_key` ASC),
  INDEX `dim_film_fact_rental_fk` (`film_key` ASC),
  INDEX `dim_customer_fact_rental_fk`(`customer_key` ASC),
  INDEX `dim_rental_date_fact_rental_fk`( `rental_date_key`  ASC),
INDEX `dim_return_date_fact_rental_fk`(`return_date_key` ASC),
  CONSTRAINT `dim_store_fact_rental_fk`
    FOREIGN KEY (`store_key`)
    REFERENCES `sakila_snowflake`.`dim_store` (`store_key`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `dim_staff_fact_rental_fk`
    FOREIGN KEY (`staff_key`)
    REFERENCES `sakila_snowflake`.`dim_staff` (`staff_key`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `dim_film_fact_rental_fk`
    FOREIGN KEY (`film_key`)
    REFERENCES `sakila_snowflake`.`dim_film` (`film_key`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,    
  CONSTRAINT `dim_customer_fact_rental_fk`
    FOREIGN KEY (`customer_key`)
    REFERENCES `sakila_snowflake`.`dim_customer`(`customer_key`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
CONSTRAINT `dim_rental_date_fact_rental_fk`
    FOREIGN KEY (`rental_date_key`)
    REFERENCES `sakila_snowflake`.`dim_date`(`date_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
CONSTRAINT `dim_return_date_fact_rental_fk`
    FOREIGN KEY (`return_date_key`)
    REFERENCES `sakila_snowflake`.`dim_date`(`date_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB
DEFAULT CHARACTER SET = utf8; '''
cursor.execute(sql)
#commit
db.commit()
# disconnect from server
db.close()

In [ ]:
# Q2 b) DML Assumption  dollaramount = (number of days between rental date and return date) * rental rate
# dollar amount is set to be zero when return_day_key is Null
#  “count_returns” are set to be zero when return_day_key is Null
# and “count_rentals” are set to be zero when rental_day_key is Null

db = pymysql.connect("localhost","root","rootroot","sakila" )
# prepare a cursor object using cursor() method
cursor = db.cursor()
# execute SQL query using execute() method.
sql ='''INSERT INTO sakila_snowflake.fact_rental
(rental_id, rental_last_update, customer_key, staff_key, film_key, store_key ,rental_date_key,
return_date_key, count_rentals, count_returns, rental_duration,dollar_amount)
SELECT 
    r.rental_id,
    r.last_update,
    c.customer_key,
    s1.staff_key,
    f.film_key,
    s2.store_key,
    date_id,
    Null,
    #CASE WHEN DATE(r.return_date) = d.date THEN d.date_id ELSE NULL END,
    CASE WHEN  r.rental_date IS NULL THEN 0 ELSE 1 END,
    CASE WHEN  r.return_date IS NULL THEN 0 ELSE 1 END,
    CASE WHEN  r.rental_date IS NULL OR r.return_date IS NULL 
    THEN NULL ELSE DATEDIFF(r.return_date,r.rental_date) END,
    CASE WHEN  r.rental_date IS NULL OR r.return_date IS NULL 
    THEN NULL ELSE DATEDIFF(r.return_date,r.rental_date)*f.film_rental_rate END
FROM
    sakila.rental r, dim_customer c, dim_film f, dim_staff s1, dim_store s2, 
    dim_date d, sakila.inventory i
WHERE
r.customer_id = c.customer_id
AND r.inventory_id = i.inventory_id
AND r.inventory_id = i.inventory_id
AND i.film_id = f.film_id
AND r.staff_id = s1.staff_id
AND s1.staff_store_id = s2.store_id
AND DATE(r.rental_date)= d.date;

UPDATE fact_rental fr
SET fr.return_date_key =(SELECT d.date_id
FROM dim_date d INNER JOIN sakila.rental r on DATE(r.return_date) = d.date 
WHERE r.rental_id = fr.rental_id);
UPDATE fact_rental 
SET dollar_amount = 0
WHERE dollar_amount IS NULL ; '''
cursor.execute(sql)
#commit
db.commit()
# disconnect from server
db.close()


In [8]:
# Q2 b)
sql = "select * from fact_rental;"
qry(sql).head()

,rental_id,rental_last_update,customer_key,staff_key,film_key,store_key,rental_date_key,return_date_key,count_returns,count_rentals,rental_duration,dollar_amount
0,1,2006-02-15 21:30:53,48,1,442,2,143,145.0,1,1,2.0,5.98
1,2,2006-02-15 21:30:53,257,1,711,2,143,147.0,1,1,4.0,11.96
2,3,2006-02-15 21:30:53,483,1,152,2,143,151.0,1,1,8.0,23.92
3,4,2006-02-15 21:30:53,26,2,665,1,143,153.0,1,1,10.0,9.90
4,5,2006-02-15 21:30:53,431,1,157,2,143,152.0,1,1,9.0,26.91
